In [1]:
import carla 
import math 
import random 
import time 
import numpy as np
    
import cv2
import open3d as o3d
from matplotlib import cm
from queue import Queue
from queue import Empty
# Connect the client and set up bp library and spawn point
client = carla.Client('localhost', 2000)
world = client.get_world()
bp_lib = world.get_blueprint_library() 
spawn_points = world.get_map().get_spawn_points()
# tm = client.get_trafficmanager(8000)



Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
#Load Specific Town Map
client.load_world('Town03')
spawn_points = world.get_map().get_spawn_points()
original_settings = world.get_settings()

In [3]:
#generate simulated Semantic Lidar, spawn near 4 Way intersection (Traffic Light 1)
#Location(x=-11.506108, y=-125.105507, z=0.152402)
def get_semantic_lidar_bp():
    lidar_bp = world.get_blueprint_library().find('sensor.lidar.ray_cast_semantic')
    
    lidar_bp.set_attribute('channels', '64')
    lidar_bp.set_attribute('range', '120.0')
    lidar_bp.set_attribute('points_per_second', '1310720')
    lidar_bp.set_attribute('rotation_frequency', '20.0')
    lidar_bp.set_attribute('upper_fov', '16.6')
    lidar_bp.set_attribute('lower_fov', '-16.6')
    lidar_bp.set_attribute('horizontal_fov', '360.0')
    lidar_bp.set_attribute('sensor_tick', '0.0')
    
    
    return lidar_bp


In [4]:
def spawn_camera():
    camera_bp = world.get_blueprint_library().find('sensor.camera.instance_segmentation')
    location = carla.Location(18,-124,13)
    rotation = carla.Rotation(-30,-133,0)
    camera_init_trans = carla.Transform(location,rotation)
    return camera_bp,camera_init_trans
    

In [5]:
def camera_callback(image, data):
    data['image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
    image.save_to_disk(f"tutorial/camera_data/{image.frame}.png")

In [6]:
#Move Spectator to Semantic Lidar location, used for checking spawn location
def setSpectator(transform):
    spectator = world.get_spectator()
    # transform = carla.Transform(semantic_lidar.get_transform().location, semantic_lidar.get_transform().rotation)
    spectator.set_transform(transform)
    print(spectator.id)

In [7]:
def turnOnVehicleAI(tmanag):
    tm_port = tmanag.get_port()
    for v in world.get_actors().filter('*vehicle*'):
        v.set_autopilot(True,tm_port)

In [8]:
def turnOffVehicleAI():
    for v in world.get_actors().filter('*vehicle*'):
        v.set_autopilot(False)
        v.destroy()

In [9]:
def sensor_callback1(sensor_data, sensor_queue, sensor_name):
    # Do stuff with the sensor_data data like save it to disk
    # Then you just need to add to the queue
    sensor_queue.put_nowait(sensor_data)

In [10]:
def sensor_callback2(sensor_data, sensor_queue, sensor_name):
    # Do stuff with the sensor_data data like save it to disk
    # Then you just need to add to the queue
    sensor_queue.put_nowait(sensor_data)

In [11]:
def transform_points(self, points, xyz, rpy):
        x, y, z = xyz[0],xyz[1],xyz[2]
        roll, pitch, yaw = rpy[0],rpy[1],rpy[2]
        pos = points[:,:3]
        M = euler_matrix(roll, pitch, yaw).astype(np.float32)
        M[0:3,3] = np.array([x, y, z])
        pos = np.hstack((pos, np.ones((pos.shape[0],1), dtype=np.float32)))
        pos = (M @ pos.T).T
        points[:,:3] = pos[:,:3]
        return points

In [1]:
def get_camera_semantic_data(data, file):
    camera_ID_list = []
    #Iterate through all pixels in image
    for color in data:
        #If pixel has tag for 
        if color.x == 4 or color.x == 10:
            
            camera_ID_list.append([color.r,color.g,color.b,color.a])
            

In [16]:
#Code to Run simulation for 30 seconds at 10 ticks/seconds
#Each tick captures Lidar data, and exports data as .ply and .csv files 
#for pointcloud, and detected objects location/bounding box info
original_settings = world.get_settings()
settings = world.get_settings()
# settings.substepping = True
# settings.max_substep_delta_time = 0.01
# settings.max_substeps = 10
settings.fixed_delta_seconds = 0.05
tm = client.get_trafficmanager(8000)
# tm.set_global_distance_to_leading_vehicle(2.5)
# tm.set_respawn_dormant_vehicles(True)
tm.set_synchronous_mode(True)

#Spawn Lidar
lidar_bp = get_semantic_lidar_bp()
lidar_location = carla.Location(18,-124,13)  #Top of lightpost
lidar_rotation = carla.Rotation(-30,-133,0)
lidar_init_trans = carla.Transform(lidar_location,lidar_rotation)
semantic_lidar1 = world.spawn_actor(lidar_bp, lidar_init_trans, attach_to=None)
time.sleep(0.5)
setSpectator(lidar_init_trans)
#Set Spectator at Lidar position

#(x=-12.215067, y=-149.450333, z=10.452678), Rotation(pitch=-18.124172, yaw=45.274269, roll=0.000071))

camera_bp, camera_init_trans = spawn_camera()
camera = world.spawn_actor(camera_bp,camera_init_trans,attach_to=None)

image_width = camera_bp.get_attribute("image_size_x").as_int()
image_height = camera_bp.get_attribute("image_size_y").as_int()
camera_data = {'image': np.zeros((image_height,image_width, 4))}

time.sleep(0.5)



settings.synchronous_mode = True

#settings.fixed_delta_seconds = 0.1
world.apply_settings(settings)

#Turn on all vehicle AI
for vehicle in world.get_actors().filter('*vehicle*'):
    vehicle.set_autopilot(True)


# Set up the AI controller for the pedestrian.... see below
# controller_bp = world.get_blueprint_library().find('controller.ai.walker')
# controller = world.spawn_actor(controller_bp, pedestrian.get_transform(), pedestrian)

# Start the controller and give it a random location to move to
# controller.start()
# controller.go_to_location(get_random_pedestrian_location())

#counting variable
t = 1

sensorQueue1 = Queue()


#setup callback to push data to queue when available
semantic_lidar1.listen(lambda data1: sensorQueue1.put_nowait(data1))
camera.listen(lambda image: camera_callback(image, camera_data))
world.tick()

while True:
    #Stop after 300 succesful data collections
    if t> 200:
        break
    #
    # if sensorQueue1.empty():
    #     time.sleep(0.05)
    #     continue
    #retrieve first in queue Lidar Data
    data1 = sensorQueue1.get(True)
    
    time.sleep(0.5)
    #save .ply file
    data1.save_to_disk('tutorial/new_lidar_output/lidar1/%.6d.ply' % data1.frame)
    
    #create .csv
    lidarFilename = "tutorial/new_lidar_output/" + str(data1.frame) + ".csv"
    lidarFile = open(lidarFilename, "w")
    lidarFile.write("Object ID,Semantic Tag,X,Y,Z,Yaw,Length,Width,Height")
    lidarFile.write('\n')
    
    id_list = []
    for detection in data1:
        try:
            if detection.object_idx not in id_list and detection.object_idx != world.get_spectator().id:
                #add id to unique list
                id_list.append(detection.object_idx)
                
                #temp actor assigned for ease of coding
                actor = world.get_actor(detection.object_idx)
    
                #write object ID to first column of csv
                lidarFile.write(str(detection.object_idx))
                lidarFile.write(',')
                lidarFile.write(str(actor.semantic_tags[0]))
                lidarFile.write(',')
                #Write Transform information for found object
                transform = actor.get_transform()
                lidarFile.write(str(transform.location.x))
                lidarFile.write(',')
                lidarFile.write(str(transform.location.y))
                lidarFile.write(',')
                lidarFile.write(str(transform.location.z))
                lidarFile.write(',')
                lidarFile.write(str(transform.rotation.yaw))
                lidarFile.write(',')
                
                #Write Bounding Box information for found object
                bounding = actor.bounding_box
                # x = bounding.location.x
                # y = bounding.location.y
                # z = bounding.location.z
                # yaw = bounding.rotation.yaw
                
                length = bounding.extent.x*2
                width = bounding.extent.y*2
                height = bounding.extent.z*2
                # file.write(str(x))
                # file.write(',')
                # file.write(str(y))
                # file.write(',')
                # file.write(str(z))
                # file.write(',')
                # file.write(str(yaw))
                # file.write(',')
                lidarFile.write(str(length))
                lidarFile.write(',')
                lidarFile.write(str(width))
                lidarFile.write(',')
                lidarFile.write(str(height))
                               
                #make new line for next ID
                lidarFile.write('\n')
        except:
            continue
    lidarFile.close()
    time.sleep(0.01)
    #iterate counting variable and step
    t+=1
    world.tick(1.0)

turnOffVehicleAI()
semantic_lidar1.stop()
semantic_lidar1.destroy()
camera.destroy()
#Return Carla to default settings after sensor grab
world.apply_settings(original_settings)

24


2456

In [21]:
semantic_lidar1.stop()
semantic_lidar1.destroy()
camera.stop()
camera.destroy()

RuntimeError: trying to operate on a destroyed actor; an actor's function was called, but the actor is already destroyed.

In [12]:
vehicle_spawn_points = [carla.Transform(carla.Location(0.700499, -189.727951, 0.039138),carla.Rotation(0,91.413528,0)),
                       carla.Transform(carla.Location(0, -161.2, 0.003371),carla.Rotation(0,91.435211,0)),
                       carla.Transform(carla.Location(5.778288, -113.229492, 0.067768),carla.Rotation(0,-88.876099,0)),
                       carla.Transform(carla.Location(-3.098442, -183.513992, -0.006345),carla.Rotation(-0.003415,91.413528,0)),
                       carla.Transform(carla.Location(-24.311588, -135.292282, -0.006437),carla.Rotation(-0.000061,1.226963,0.001293)),
                       carla.Transform(carla.Location(9.184540, -101.743164, 0.000199),carla.Rotation(0,-88.586418,0)),
                       carla.Transform(carla.Location(73.203979, -136.704651, 7.996281),carla.Rotation(-1.480286,-178.773956,-0.116272)),
                       carla.Transform(carla.Location(73.266441, -139.972733, 8.045337),carla.Rotation(-1.480286,-178.773956,-0.116272)),
                       carla.Transform(carla.Location(-15.723951, -131.572281, 0.279280),carla.Rotation(0,1.2,0)),
                       carla.Transform(carla.Location(10.139042, -146.582550,-0.007044),carla.Rotation(0.002227,-88.586418,-0.000366)),
                       carla.Transform(carla.Location(16.879513, -134.409973,0.066324),carla.Rotation(1.567317,1.230288,0.030370)),
                       carla.Transform(carla.Location(-4.364792, -126.337173,0.000056),carla.Rotation(0,91.413528,0)),
                       carla.Transform(carla.Location(-11.102114, -138.510193,-0.026897),carla.Rotation(0,-178.772690,0))]                       

In [15]:
#Spawn Vehicles
spawn_points = world.get_map().get_spawn_points()
for i in range(6):
    vehicle_bp = random.choice(bp_lib.filter('vehicle'))
    world.try_spawn_actor(vehicle_bp, random.choice(vehicle_spawn_points))

In [134]:
#Spawn Vehicles
spawn_points = world.get_map().get_spawn_points()
for i in range(50):
    vehicle_bp = random.choice(bp_lib.filter('vehicle'))
    npc = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))

In [19]:
for vehicle in world.get_actors().filter('*vehicle*'):
    vehicle.destroy()

In [14]:
original_settings = world.get_settings()
settings = world.get_settings()
# settings.substepping = True
# settings.max_substep_delta_time = 0.01
# settings.max_substeps = 10
settings.fixed_delta_seconds = 0.05
tm = client.get_trafficmanager(8000)
# tm.set_global_distance_to_leading_vehicle(2.5)
# tm.set_respawn_dormant_vehicles(True)
# tm.set_synchronous_mode(True)
settings.synchronous_mode = True
#Spawn Lidar
lidar_bp = get_semantic_lidar_bp()
lidar_location = carla.Location(18,-124,13)  #Top of lightpost
lidar_rotation = carla.Rotation(-30,-133,0)
lidar_init_trans = carla.Transform(lidar_location,lidar_rotation)
semantic_lidar1 = world.spawn_actor(lidar_bp, lidar_init_trans, attach_to=None)
time.sleep(0.5)
setSpectator(lidar_init_trans)
#Set Spectator at Lidar position

#(x=-12.215067, y=-149.450333, z=10.452678), Rotation(pitch=-18.124172, yaw=45.274269, roll=0.000071))
lidar_bp = get_semantic_lidar_bp()
lidar_location = carla.Location(-12.2,-149.5,13)  #Top of lightpost
lidar_rotation = carla.Rotation(-18,45,0)
lidar_init_trans = carla.Transform(lidar_location,lidar_rotation)
# semantic_lidar2 = world.spawn_actor(lidar_bp, lidar_init_trans, attach_to=None)


sensorQueue1 = Queue()
sensorQueue2 = Queue()
semantic_lidar1.listen(lambda data: sensorQueue1.put_nowait(data))
# semantic_lidar2.listen(lambda data: sensorQueue2.put_nowait(data))
camera.listen(lambda image: camera_callback(image, camera_data))
time.sleep(0.5)
#settings.fixed_delta_seconds = 0.1
world.apply_settings(settings)
world.tick()
for v in world.get_actors().filter('*vehicle*'):
    v.set_autopilot(True)
for i in range(100):
    world.tick()
    time.sleep(0.1)
for v in world.get_actors().filter('*vehicle*'):
    v.set_autopilot(False)
semantic_lidar1.destroy()
semantic_lidar2.destroy()
world.apply_settings(original_settings)

24


NameError: name 'semantic_lidar2' is not defined

In [19]:
location = carla.Location(16,-150,1.18)
rotation = carla.Rotation(0,0,0)
spectator = world.get_spectator()
spectator.set_transform(carla.Transform(location, rotation))

In [58]:
pedestrian_spawns = [[-13,-124,1.18],
                     [16,-145,1.18],
                     [-13,-145,1.198]]

In [37]:
for spawn in pedestrian_spawns:
    location = carla.Location(spawn[0],spawn[1],spawn[2])
    rotation = carla.Rotation(0,0,0)
    spectator.set_transform(carla.Transform(location,rotation))
    time.sleep(3)

In [59]:
pedestrian_bp = random.choice(world.get_blueprint_library().filter('*walker.pedestrian*'))
location = random.choice(pedestrian_spawns)
transform = carla.Transform(carla.Location(location[0],location[1],location[2]))
pedestrian = world.try_spawn_actor(pedestrian_bp, transform)

In [6]:
spectator = world.get_spectator()
transform = pedestrian.get_transform()
spectator.set_transform(transform)

NameError: name 'pedestrian' is not defined

In [60]:
def get_random_pedestrian_location():
    location = random.choice(pedestrian_spawns)
    carla_location = carla.Location(location[0],location[1],location[2])
    return carla_location

In [29]:
# Set up the AI controller for the pedestrian.... see below
controller_bp = world.get_blueprint_library().find('controller.ai.walker')
controller = world.spawn_actor(controller_bp, pedestrian.get_transform(), pedestrian)

# Start the controller and give it a random location to move to
controller.start()
controller.go_to_location(get_random_pedestrian_location())

In [10]:
settings = world.get_settings()
settings.fixed_delta
settings.synchronous_mode = True
world.apply_settings(settings)

world.tick()
#turn all vehicle auto pilot on
for v in world.get_actors().filter('*vehicle*'):
    v.set_autopilot(True)

for i in range(100):
    world.tick()
#Turn all vehicle auto pilot off
for v in world.get_actors().filter('*vehicle*'):
    v.set_autopilot(False)

settings.synchronous_mode = False
world.apply_settings(settings)

2745

In [14]:
#Turn all vehicle auto pilot off
for v in world.get_actors().filter('*vehicle*'):
    v.set_autopilot(False)

In [43]:
#turn all vehicle auto pilot on
for v in world.get_actors().filter('*vehicle*'):
    v.set_autopilot(True)

In [71]:
#Destroy all walkers
for i in world.get_actors().filter('*walker*'):
    world.get_actor(i.id).destroy()

ERROR: failed to destroy actor 345 : unable to destroy actor: not found 


In [74]:
#Destroy all vehicles
for i in world.get_actors().filter('*vehicle*'):
    world.get_actor(i.id).destroy()

In [3]:
#Spawn Licoln MKZ at random spawn point
vehicle_bp = bp_lib.find('vehicle.lincoln.mkz_2020')
vehicle = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))

In [4]:
#Move spectator camera to behind and above spawned vehicle
spectator = world.get_spectator()
transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-4,z=2.5)), vehicle.get_transform().rotation)
spectator.set_transform(transform)

In [25]:
world.get_spectator().set_transform(world.get_actor(392).get_transform())

In [135]:
#List all actors with IDs
for v in world.get_actors().filter('*sprint*'):
    print(str(v.id) + ' ' + str(v.type_id))

1339 vehicle.mercedes.sprinter
1488 vehicle.mercedes.sprinter
1441 vehicle.mercedes.sprinter
1414 vehicle.mercedes.sprinter


In [10]:
print(world.get_actor(143).semantic_tags[0])

[10, 4]


In [11]:
bp = random.choice(bp_lib.filter('walker'))
transform = world.get_map().get_spawn_points()[0]
pedestrain = world.spawn_actor(bp, transform)

In [13]:
spectator = world.get_spectator()
transform = carla.Transform(pedestrain.get_transform().transform(carla.Location(x=-4,z=2.5)), vehicle.get_transform().rotation)
spectator.set_transform(transform)

In [17]:
spectator = world.get_spectator()
transform = carla.Transform(carla.Location(0,10,10),carla.Rotation(0,0,0))
spectator.set_transform(transform)
                            

In [136]:
world.get_spectator().set_transform(world.get_actor(1488).get_transform())

In [137]:
print(world.get_actor(1488).get_transform())

Transform(Location(x=-11.102114, y=-138.510193, z=-0.026897), Rotation(pitch=0.000000, yaw=-178.772690, roll=0.000000))


In [103]:
print(world.get_spectator().get_transform())

Transform(Location(x=-15.723951, y=-131.572281, z=0.279280), Rotation(pitch=-11.511990, yaw=-2.386841, roll=-0.000580))
